In [2]:
# https://github.com/thelokeshgoel00/youtube-reinforcement/blob/master/Reinforcement%20L1/ReInforcementLearningInCartPole.ipynb
import numpy as np
import gym
from IPython.display import clear_output


In [3]:
env = gym.make("CartPole-v1")

In [3]:
env.action_space.n

2

In [4]:
def random_games():
    for episode in range(10):
        env.reset()
        
        for t in range(500):
            env.render()
            action=env.action_space.sample()
            next_state,reward,done,info=env.step(action)
#             print(t,next_state,done,info,action)
            
            if done:
                break
    env.close()

random_games()
            
    
     

In [108]:
def gather_data(env):
    num_trials = 10000
    min_score = 100
    sim_steps = 300
    trainingX,trainingY = [],[]
    scores = []
    for trial in range(num_trials):
        observation = env.reset()
        score = 0
        training_sampleX,training_sampleY = [],[]        
        
        for step in range(sim_steps):          
#             env.render()
            action = np.random.randint(0,2) # left or right
            one_hot_action = np.zeros(2)
            one_hot_action[action] = 1
            training_sampleX.append(observation)
            training_sampleY.append(one_hot_action)
            observation , reward, done, info = env.step(action)
            score += reward
            if done:
                break
            
        if score>min_score:
            scores.append(score)
            trainingX+=training_sampleX
            trainingY+=training_sampleY
            
        clear_output(wait=True)
        print("Episodes: ",trial)
        
    trainingX,trainingY = np.array(trainingX), np.array(trainingY)
    print("Average: {}".format(np.mean(scores)))
    print("Median: {}".format(np.median(scores)))
#     env.close()
    return trainingX,trainingY

In [109]:
x_train,y_train=gather_data(env)


Episodes:  9978


In [110]:
print(x_train.shape)

(416, 4)


In [111]:
TRAIN_SIZE=400
xx_train,xx_test=x_train[:TRAIN_SIZE],x_train[TRAIN_SIZE:]
yy_train,yy_test=y_train[:TRAIN_SIZE],y_train[TRAIN_SIZE:]
print(yy_train.shape)

(400, 2)


In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam

In [119]:
def create_model():
    model = Sequential()
    model.add(Dense(256,input_shape=(4,),activation='relu'))
    model.add(Dropout(0.5))   

    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.5))   
    

    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2, activation='linear'))
    

    model.compile(loss='mse',optimizer=Adam(lr=0.01),metrics=['accuracy'])
    return model

In [120]:
model = create_model()
model.fit(xx_train,yy_train,epochs=20)    
    

Train on 400 samples
Epoch 1/20
400/400 [==============================] - 1s 3ms/sample - loss: 0.4930 - accuracy: 0.5100
Epoch 2/20
400/400 [==============================] - 0s 285us/sample - loss: 0.2883 - accuracy: 0.4900
Epoch 3/20
400/400 [==============================] - 0s 297us/sample - loss: 0.2599 - accuracy: 0.5550
Epoch 4/20
400/400 [==============================] - 0s 315us/sample - loss: 0.2688 - accuracy: 0.5775
Epoch 5/20
400/400 [==============================] - 0s 295us/sample - loss: 0.2520 - accuracy: 0.5625
Epoch 6/20
400/400 [==============================] - 0s 310us/sample - loss: 0.2560 - accuracy: 0.5200
Epoch 7/20
400/400 [==============================] - 0s 292us/sample - loss: 0.2572 - accuracy: 0.5425
Epoch 8/20
400/400 [==============================] - 0s 297us/sample - loss: 0.2466 - accuracy: 0.5625
Epoch 9/20
400/400 [==============================] - 0s 310us/sample - loss: 0.2485 - accuracy: 0.6000
Epoch 10/20
400/400 [========================

In [121]:
model.evaluate(xx_test,yy_test)

16/16 [==============================] - 0s 10ms/sample - loss: 0.2325 - accuracy: 0.5625


[0.23254814743995667, 0.5625]

In [125]:
num_trials = 3


for trial in range(num_trials):
    observation = env.reset()
    score = 0
    done=False
    steps=0
    
    while not done:        
        env.render()
        predict=model.predict(observation.reshape(-1,4))
        action = np.argmax(predict)
        observation,reward,done,info = env.step(action)
        steps+=1
        score+=reward
    print("Steps ",steps," Score ",score)

env.close()


Steps  239  Score  239.0
Steps  305  Score  305.0
Steps  348  Score  348.0


# Using DOT  (100% accuracy)

## https://www.youtube.com/watch?v=Bi-CKm9zS9c

In [129]:
best_length=0
episode_lengths=[]

best_weight=np.zeros(4)

for agent in range(100):
    new_weight=np.random.uniform(-1.0,1.0,4)
    
    length=[]
    for game in range(100):
        observation=env.reset()
        done=False
        step=0
        
        while not done:
            step+=1
            action=1 if np.dot(observation,new_weight)>0 else 0
            observation,reward,done,_=env.step(action)
            
            if done:
                break
        length.append(step)
    average_length=float(sum(length)/len(length))
    
    if average_length>best_length:
        best_length=average_length
        best_weight=new_weight
    episode_lengths.append(average_length)
    if agent%10==0:
        print("Best length: ",best_length)

            
            
        


Best length:  63.81
Best length:  237.13
Best length:  237.13
Best length:  301.51
Best length:  301.51
Best length:  301.51
Best length:  301.51
Best length:  301.51
Best length:  301.51
Best length:  500.0


# Record as a Video

In [130]:
# for recording a video, need to install ffmpeg :   conda install -c conda-forge ffmpeg
from gym import wrappers

In [131]:
done=False
cnt=0
env=wrappers.Monitor(env,"Video",force=True)
observation=env.reset()

while not done:
    action=1 if np.dot(observation,best_weight)>0 else 0
    observation,reward,done,_=env.step(action)
    
    cnt+=1
    if done:
        break
        
env.close()
    
print("Game lasted ",cnt)

Game lasted  500
